# Concentration of free barbed ends

In [20]:
import numpy as np

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.models import CustomJS, Slider
from bokeh.layouts import row, widgetbox

# Initialize plot.
WZero = 1
kNuc = 10
kPol = 100
kCap = 1
kActin = 10
N = 100000
dt = 1e-4
t = np.arange(0, N * dt, dt)
E = np.zeros(N)
WA = np.zeros(N)
E[0] = 0.01
# Iterate.
for i in range(0, N - 1):
    dEdt = kNuc * E[i] * WA[i] - kCap * E[i]
    dWAdt = -kNuc * E[i] * WA[i] - kPol * E[i] * WA[i] + kActin * (WZero - WA[i])
    E[i + 1] = E[i] + dEdt * dt
    WA[i + 1] = WA[i] + dWAdt * dt

source = ColumnDataSource(data=dict(x=t, y=E))

p = figure(title="Concentration of free barbed ends", y_range=(0, 2), plot_width=600, plot_height=600)
p.xaxis[0].axis_label = 'Time (s)'
p.yaxis[0].axis_label = 'Concentration'
p.line('x', 'y', source=source, line_width=2)

# Callback
callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var kNuc = nuc_rate.value;
    var kPol = pol_rate.value;
    var kCap = cap_rate.value;
    var kActin = load_rate.value;
    var WZero = 1;
    var N = 100000;
    var dt = 1e-4;
    var WA = 0;
    var dEdt = 0;
    var dWAdt = 0;
    x = data['x']
    y = data['y']
    y[0] = 0.01;
    for (i = 0; i < x.length; i++) {
        dEdt = kNuc * y[i] * WA - kCap * y[i];
        dWAdt = -kNuc * y[i] * WA - kPol * y[i] * WA + kActin * (WZero - WA);
        y[i + 1] = y[i] + dEdt * dt;
        WA = WA + dWAdt * dt;
    }
    source.change.emit();
""")

# Sliders
nuc_rate_slider = Slider(start=0, end=100, value=10, step=0.1, title="Nucleation rate", callback=callback)
callback.args["nuc_rate"] = nuc_rate_slider
pol_rate_slider = Slider(start=0, end=1000, value=100, step=0.1, title="Polymerization rate", callback=callback)
callback.args["pol_rate"] = pol_rate_slider
cap_rate_slider = Slider(start=0, end=10, value=1, step=0.1, title="Capping rate", callback=callback)
callback.args["cap_rate"] = cap_rate_slider
load_rate_slider = Slider(start=0, end=100, value=10, step=0.1, title="Loading rate", callback=callback)
callback.args["load_rate"] = load_rate_slider

# Plot
layout = row(p, widgetbox(nuc_rate_slider, pol_rate_slider, cap_rate_slider, load_rate_slider),)
output_notebook()
show(layout)

Loading BokehJS ...